### **TASK 2**
##**Create a simple tool that will be able to extract TCR sequences from bulk-RNAseq**

###We shall be using MiXCR owing to its superior features over TRUST4

# Let's start by connecting our drive

In [1]:
from google.colab import drive
#Mounting Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

Mounted at /content/drive
Google Drive mounted successfully!


#Installing essential pacakges

In [2]:
!pip install -q condacolab
!pip install matplotlib-venn

#Importing all necessary packages

In [1]:
import re
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import subprocess
from pathlib import Path
from matplotlib_venn import venn3
from functools import reduce
import shutil
import sys
import os
# Parallel Processing with Multiprocessing
import gzip
from multiprocessing import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import shutil
import subprocess
from pathlib import Path
import multiprocessing as mp
import glob, math
import condacolab
import shlex

#Please refer condacolab's documentation for installtion
#[Here...](https://pypi.org/project/condacolab/)

In [4]:
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [2]:
condacolab.check()

✨🍰✨ Everything looks OK!


#Add conda to the path

In [3]:
conda_path = '/usr/local/bin'
os.environ['CONDA_EXE'] = os.path.join(conda_path, 'conda')
os.environ['PATH'] = f"{conda_path}:{os.environ['PATH']}"
sys.path.insert(0, os.path.join(conda_path, '..', 'lib', 'python3.10', 'site-packages'))

#Steps to install MiXCR
#**You will need to get a free locense to run this package**
#Please follow the link here
[License Link](https://mixcr.com/mixcr/getting-started/milm/)

In [7]:
# Installation Steps for MiXCR
%cd /
%cd /usr/local/bin
!wget https://github.com/milaboratory/mixcr/releases/download/v4.6.0/mixcr-4.6.0.zip
!unzip mixcr-4.6.0.zip

/
/usr/local/bin
--2025-09-02 18:41:57--  https://github.com/milaboratory/mixcr/releases/download/v4.6.0/mixcr-4.6.0.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/33134597/6bdd7a80-9aea-4572-b796-6065ef5d1f8b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-02T19%3A31%3A33Z&rscd=attachment%3B+filename%3Dmixcr-4.6.0.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-02T18%3A31%3A03Z&ske=2025-09-02T19%3A31%3A33Z&sks=b&skv=2018-11-09&sig=7e3QhLnl%2Fzp6NwNx4qmp%2BIrtR11HkFQ1J2ZO5M48cJ4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NjgzODcwOCwibmJmIjoxNzU2ODM4NDA4LCJwYXRoIjoicmVsZWFzZWFzc

In [8]:
# Setting up license key for MiXCR [Add this in our OS environment]
os.environ['MI_LICENSE'] = 'E-UZCNDGRBYRVXJUSGMPFWBGRMAGYVBGJHUSXMTPRSHWVCSOGL'

In [9]:
# verify if it was installed correclty
!mixcr -h

Usage: mixcr [-h] [-v] [COMMAND]
  -h, --help      Show this help message and exit.
  -v, --version   Print version information and exit
Base commands:
  analyze            Run full MiXCR pipeline for specific input.
  align              Builds alignments with V,D,J and C genes for input sequencing reads.
  refineTagsAndSort  Applies error correction algorithm for tag sequences and sorts resulting file
                       by tags.
  assemblePartial    Assembles partially aligned reads into longer sequences.
  extend             Impute alignments or clones with germline sequences.
  assemble           Assemble clones.
  assembleContigs    Assemble full sequences.
  groupClones        Group clones by cells. Required data with cell tags.
  findAlleles        Find allele variants in clnx.
  findShmTrees       Builds SHM trees.
  downsample         Downsample clonesets.
  qc                 Perform quality control checks on results.
Postanalysis commands:
  postanalysis  Run postanalysis

In [10]:
# Step 1 Pulling the data systematically from the mounted drive
base_dir = "/content/drive/MyDrive/Bio_Informatics_RA_Assignment"

patients = ["Patient1"]

patient_files = ["Patient1_merged_1.fastq.gz", "Patient1_merged_2.fastq.gz"]

# Output dir for results
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok=True)

In [11]:
# Optimizing IO
def stage_data_locally(patients, patient_files, base_dir):
    """
    Copies necessary FASTQ and reference files from Drive to the local Colab disk.
    Returns a dictionary of local FASTQ paths for each patient.
    """
    local_data_dir = "/content/local_data"
    os.makedirs(local_data_dir, exist_ok=True)

    local_paths = {}
    for patient in patients:
        fastq_dir = os.path.join(base_dir, patient)

        # Copy FASTQ files
        r1_name, r2_name = patient_files[patient]
        local_r1_path = os.path.join(local_data_dir, r1_name)
        local_r2_path = os.path.join(local_data_dir, r2_name)

        print(f"Copying {os.path.join(fastq_dir, r1_name)} to {local_r1_path}")
        shutil.copy(os.path.join(fastq_dir, r1_name), local_r1_path)
        shutil.copy(os.path.join(fastq_dir, r2_name), local_r2_path)

        local_paths[patient] = (local_r1_path, local_r2_path)

    print("All FASTQ files copied to local disk.")
    return local_paths

In [12]:
# Create a local directory to store results for efficiency
local_results_dir = "/content/local_results"
os.makedirs(local_results_dir, exist_ok=True)


# 🔧 MiXCR — Optimized Pipeline (Threading · Chunking · Caching)

This section adds a robust, Colab-friendly workflow for **MiXCR**. Key features:

- Uses **multi-threading** automatically (`-t` set to all available CPU cores).
- **Streams** from `.fastq.gz` directly (no full decompression).
- Optional **chunking** for very large FASTQs, with smart per-chunk threading.
- **Caching**: skips steps if outputs are already up-to-date.
- Clean **logging** + **reports** for reproducibility.

> ✅ Recommended free Colab runtime: **CPU** (High-RAM if available). MiXCR does **not** use GPU.


In [13]:
# State the Path first
mixcr_path = "/usr/local/bin/mixcr"

In [14]:
MIXCR_VERSION = "4.6.0"  # change if needed
BIN_DIR = "/content/bin"
os.makedirs(BIN_DIR, exist_ok=True)

def run(cmd, **kwargs):
    print("$", cmd)
    return subprocess.run(cmd, shell=True, check=True, text=True, **kwargs)
# SeqKit install (static Linux binary)
seqkit_path = os.path.join(BIN_DIR, "seqkit")
if not os.path.exists(seqkit_path):
    run("wget -q https://github.com/shenwei356/seqkit/releases/download/v2.8.2/seqkit_linux_amd64.tar.gz -O /content/seqkit.tar.gz")
    run("tar -xzf /content/seqkit.tar.gz -C /content/")
    # The tar expands to a directory with the binary inside
    found = False
    for root, dirs, files in os.walk("/content"):
        if "seqkit" in files and "linux_amd64" in root:
            shutil.copy(os.path.join(root, "seqkit"), seqkit_path)
            found = True
            break
    if not found:
        # fallback: sometimes the tar unpacks directly
        if os.path.exists("/content/seqkit"):
            shutil.copy("/content/seqkit", seqkit_path)
    run(f"chmod +x {seqkit_path}")
else:
    print("SeqKit already installed:", seqkit_path)

# Put BIN_DIR on PATH for this session
os.environ["PATH"] = f"{BIN_DIR}:" + os.environ["PATH"]
print("PATH ->", os.environ["PATH"])
run("mixcr -v || true")
run("seqkit version || true")
print("\n✅ Tooling ready.")

$ wget -q https://github.com/shenwei356/seqkit/releases/download/v2.8.2/seqkit_linux_amd64.tar.gz -O /content/seqkit.tar.gz
$ tar -xzf /content/seqkit.tar.gz -C /content/
$ chmod +x /content/bin/seqkit
PATH -> /content/bin:/usr/local/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
$ mixcr -v || true
$ seqkit version || true

✅ Tooling ready.


#Helper Functions

In [15]:
def cpu_threads():
  try:
    n = mp.cpu_count() or 2
  except Excetpion as e:
    n = 2
  return max(1, n)

def newer_than(output, *inputs):
    out = Path(output)
    if not out.exists():
        return False
    out_mtime = out.stat().st_mtime
    for i in inputs:
        p = Path(i)
        if not p.exists():
            return False
        if p.stat().st_mtime > out_mtime:
            return False
    return True

def sh(cmd):
    print(f"$ {cmd}")
    # stream output live
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in proc.stdout:
        print(line, end="")
    rc = proc.wait()
    if rc != 0:
        raise RuntimeError(f"Command failed with exit code {rc}: {cmd}")
    return rc

def _run(cmd, cwd=None):
    """Run a shell command with full error surfacing."""
    print("$", " ".join(shlex.quote(c) for c in cmd))
    res = subprocess.run(cmd, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if res.returncode != 0:
        raise subprocess.CalledProcessError(res.returncode, cmd, output=res.stdout, stderr=res.stderr)
    return res

def _pair_chunk_files(r1_dir, r2_dir):
    r1 = sorted(list(Path(r1_dir).glob("*.fastq.gz")) + list(Path(r1_dir).glob("*.fq.gz")))
    r2 = sorted(list(Path(r2_dir).glob("*.fastq.gz")) + list(Path(r2_dir).glob("*.fq.gz")))
    if len(r1) != len(r2):
        raise ValueError(f"R1/R2 chunk mismatch: {len(r1)} vs {len(r2)}")
    return list(zip(r1, r2))

def _choose_parallelism(n_chunks, total_threads):
    """
    Budget threads safely: threads_per_job * jobs <= total_threads.
    Prefer ~2–4 threads per chunk if possible.
    """
    total_threads = max(1, int(total_threads or (os.cpu_count() or 1)))
    if n_chunks <= 1:
        return 1, total_threads  # 1 job, all threads

    # start with 4 threads per job if we can afford it
    t_per = min(4, total_threads)
    jobs = max(1, min(n_chunks, total_threads // t_per))
    if jobs == 0:
        jobs = 1
        t_per = total_threads
    # recompute t_per to fully use available threads
    t_per = max(1, total_threads // jobs)
    return jobs, t_per

def _align_one_chunk(mixcr_path, preset, species, reads_layout, r1, r2, out_prefix, threads, report_path):
    """
    Faithful to your working single-run flags.
    mixcr align --preset rna-seq --species hsa -OreadsLayout=Collinear --threads N R1 R2 out.vdjca
    """
    align_cmd = [
        mixcr_path, "align",
        "--preset", preset,
        "--species", species,
        "-OreadsLayout=" + reads_layout,
        "--threads", str(threads),
        "--report", str(report_path),
        str(r1), str(r2),
        str(out_prefix) + ".vdjca",
    ]
    _run(align_cmd)
    return str(out_prefix) + ".vdjca"

#MiXCR Core Steps (with caching)

In [16]:
def mixcr_align(r1, r2, out_vdjca, report_path=None, threads=None, extra_args=None):
    r1, r2, out_vdjca = map(str, (r1, r2, out_vdjca))
    threads = threads or cpu_threads()
    report = f"--report {report_path}" if report_path else ""
    extra = " ".join(extra_args) if extra_args else ""
    if newer_than(out_vdjca, r1, r2):
        print(f"🔁 [cache] align up-to-date: {out_vdjca}")
        return out_vdjca
    sh(f"mixcr align -t {threads} {report} {extra} {r1} {r2} {out_vdjca}")
    return out_vdjca

def mixcr_assemble(in_vdjca, out_clns, report_path=None, threads=None, extra_args=None):
    in_vdjca, out_clns = map(str, (in_vdjca, out_clns))
    threads = threads or cpu_threads()
    report = f"--report {report_path}" if report_path else ""
    extra = " ".join(extra_args) if extra_args else ""
    if newer_than(out_clns, in_vdjca):
        print(f"🔁 [cache] assemble up-to-date: {out_clns}")
        return out_clns
    sh(f"mixcr assemble -t {threads} {report} {extra} {in_vdjca} {out_clns}")
    return out_clns

def mixcr_export_clones(in_clns, out_txt, preset="full", extra_args=None):
    in_clns, out_txt = map(str, (in_clns, out_txt))
    extra = " ".join(extra_args) if extra_args else ""
    if newer_than(out_txt, in_clns):
        print(f"🔁 [cache] export up-to-date: {out_txt}")
        return out_txt
    sh(f"mixcr exportClones --preset {preset} {extra} {in_clns} {out_txt}")
    return out_txt

#Chunking Utilities (SeqKit split2) — for very large FASTQs

In [17]:
def split_fastq_gz(r1_gz, r2_gz, out_dir, chunk_reads=2_000_000, force=False, patient = ""):
    out_dir = Path(out_dir)
    r1_out = out_dir / f"{patient}chunks_R1"
    r2_out = out_dir / f"{patient}chunks_R2"
    r1_out.mkdir(parents=True, exist_ok=True)
    r2_out.mkdir(parents=True, exist_ok=True)

    # If chunks already exist and force==False, skip splitting
    existing = list(r1_out.glob("*.fq.gz")) or list(r1_out.glob("*.fastq.gz"))
    if existing and not force:
        print("🔁 [cache] chunks already exist — skipping split")
        return r1_out, r2_out

    sh(f"seqkit split2 -s {chunk_reads} -O {r1_out} {r1_gz}")
    sh(f"seqkit split2 -s {chunk_reads} -O {r2_out} {r2_gz}")
    return r1_out, r2_out

def pair_chunk_files(r1_dir, r2_dir):
    r1_dir, r2_dir = Path(r1_dir), Path(r2_dir)
    r1_files = sorted([p for p in r1_dir.glob('*.fq.gz')] + [p for p in r1_dir.glob('*.fastq.gz')])
    r2_files = sorted([p for p in r2_dir.glob('*.fq.gz')] + [p for p in r2_dir.glob('*.fastq.gz')])
    # Pair by numeric suffix in filenames (seqkit uses part_001, part_002, ...)
    def key(p):
        s = p.stem  # remove .gz
        s = s.replace(".fastq", "").replace(".fq", "")
        # extract last number group
        import re
        m = re.findall(r"(\d+)$", s)
        return int(m[-1]) if m else -1
    r1_files.sort(key=key)
    r2_files.sort(key=key)
    if len(r1_files) != len(r2_files):
        raise ValueError("R1 and R2 chunk counts differ")
    return list(zip(r1_files, r2_files))

#Accelerated per-chunk alignment (thread budgeting + caching)

In [18]:
def align_chunks(r1_dir, r2_dir, out_dir, threads_total=None, per_job_min=2, extra_args=None):
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    threads_total = threads_total or cpu_threads()
    pairs = pair_chunk_files(r1_dir, r2_dir)
    n_chunks = len(pairs)
    if n_chunks == 0:
        raise RuntimeError("No chunks found to align")

    # Determine jobs in parallel and threads per job
    # Keep per-job threads >= per_job_min, and avoid oversubscription
    max_jobs = max(1, threads_total // per_job_min)
    jobs = min(max_jobs, n_chunks)
    threads_per_job = max(per_job_min, threads_total // jobs)

    print(f"Threads total: {threads_total} | chunks: {n_chunks} | parallel jobs: {jobs} | threads/job: {threads_per_job}")

    futures = []
    results = []

    def do_align(i, r1, r2):
        out_vdjca = out_dir / f"chunk_{i:04d}.vdjca"
        rep = out_dir / f"chunk_{i:04d}_align_report.txt"
        mixcr_align(r1, r2, out_vdjca, report_path=rep, threads=threads_per_job, extra_args=extra_args)
        return str(out_vdjca)

    with ThreadPoolExecutor(max_workers=jobs) as ex:
        for i, (r1, r2) in enumerate(pairs, start=1):
            futures.append(ex.submit(do_align, i, r1, r2))
        for f in as_completed(futures):
            results.append(f.result())

    results = sorted(results)
    print(f"✅ Completed {len(results)} chunk alignments")
    return results

#Merge chunked alignments, assemble, and export


In [19]:
def merge_and_finalize(vdjca_list, merged_vdjca, clns_path, clones_txt, threads=None, export_preset="full"):
    merged_vdjca = str(merged_vdjca); clns_path = str(clns_path); clones_txt = str(clones_txt)
    # Cache-aware: if merged_vdjca exists and newer than all parts, skip merge
    def parts_newer_than(out, parts):
        if not Path(out).exists():
            return True
        out_m = Path(out).stat().st_mtime
        return any(Path(p).stat().st_mtime > out_m for p in parts)

    if parts_newer_than(merged_vdjca, vdjca_list):
        # mixcr mergeChunks requires vdjca args
        args = " ".join(map(str, vdjca_list))
        sh(f"mixcr mergeChunks {merged_vdjca} {args}")
    else:
        print(f"🔁 [cache] merged VDJCA up-to-date: {merged_vdjca}")

    mixcr_assemble(merged_vdjca, clns_path, report_path=str(Path(clns_path).with_suffix('.assemble_report.txt')), threads=threads)
    mixcr_export_clones(clns_path, clones_txt, preset=export_preset)
    return merged_vdjca, clns_path, clones_txt

#Detect read count, and choose chunk size

In [20]:
def count_reads_fastq_gz(fastq_gz):
    # Count lines / 4 = number of reads
    cmd = f"zcat {fastq_gz} | wc -l"
    lines = int(subprocess.check_output(cmd, shell=True).decode().strip())
    return lines // 4

def choose_chunk_size(total_reads, target_chunks=20, min_chunk=500_000, max_chunk=5_000_000):
    chunk = max(min_chunk, min(max_chunk, total_reads // target_chunks))
    return int(chunk)

#High Level Pipeline

In [21]:
def run_mixcr_pipeline(
    fastq1, fastq2, work_dir, sample,
    chunk_reads=None,                   # e.g., 2_000_000; if None => no chunking
    mixcr_path="mixcr",
    preset="rna-seq",
    species="hsa",
    reads_layout="Collinear",
    total_threads=None,
    force_split=False,
    reuse_if_done=True
):
    """
    Chunked MiXCR (faithful flags) → assemble once across all chunks → exportClones.
    Produces: {work_dir}/{sample}_mixcr.clones.txt

    Returns dict with output paths.
    """
    work_dir = Path(work_dir)
    work_dir.mkdir(parents=True, exist_ok=True)

    final_clns   = work_dir / f"{sample}_mixcr.clns"
    final_clones = work_dir / f"{sample}_mixcr.clones.txt"
    assemble_report = work_dir / f"{sample}_mixcr_assemble_report.txt"

    # Fast path: reuse final outputs
    if reuse_if_done and final_clones.exists() and final_clns.exists():
        print(f"🔁 [cache] Found final MiXCR outputs for {sample}, skipping.")
        return {
            "clns": str(final_clns),
            "clones": str(final_clones),
            "assemble_report": str(assemble_report),
        }

    # --- Prepare chunks (or “virtual” single chunk) ---
    if chunk_reads and int(chunk_reads) > 0:
        # Safe split with seqkit (must be installed)
        chunks_root = work_dir / "chunks"
        r1_dir = chunks_root / f"{sample}_chunks_R1"
        r2_dir = chunks_root / f"{sample}_chunks_R2"
        r1_dir.mkdir(parents=True, exist_ok=True)
        r2_dir.mkdir(parents=True, exist_ok=True)

        # If already split and not forced, reuse
        existing = list(r1_dir.glob("*.fastq.gz")) + list(r1_dir.glob("*.fq.gz"))
        if not existing or force_split:
            # seqkit split2 -s <reads>
            _run(["seqkit", "split2", "-s", str(int(chunk_reads)), "-O", str(r1_dir), str(fastq1)])
            _run(["seqkit", "split2", "-s", str(int(chunk_reads)), "-O", str(r2_dir), str(fastq2)])
        pairs = _pair_chunk_files(r1_dir, r2_dir)
    else:
        # No chunking: treat original pair as a single “chunk”
        pairs = [(Path(fastq1), Path(fastq2))]

    n_chunks = len(pairs)
    jobs, threads_per_job = _choose_parallelism(n_chunks, total_threads)
    print(f"[{sample}] Chunks: {n_chunks} | parallel jobs: {jobs} | threads/job: {threads_per_job}")

    aligned_dir = work_dir / "aligned_chunks"
    aligned_dir.mkdir(exist_ok=True, parents=True)

    # Align all chunks (parallel)
    vdjca_paths = []
    errors = []
    if n_chunks == 1:
        # single run, faithful command
        out_prefix  = aligned_dir / f"chunk_0001"
        report_file = aligned_dir / f"chunk_0001_align_report.txt"
        try:
            vdj = _align_one_chunk(mixcr_path, preset, species, reads_layout,
                                   pairs[0][0], pairs[0][1], out_prefix, threads_per_job, report_file)
            vdjca_paths.append(vdj)
        except subprocess.CalledProcessError as e:
            print(e.stderr or e.output)
            raise
    else:
        with ProcessPoolExecutor(max_workers=jobs) as ex:
            futs = {}
            for i, (r1, r2) in enumerate(pairs, start=1):
                out_prefix  = aligned_dir / f"chunk_{i:04d}"
                report_file = aligned_dir / f"chunk_{i:04d}_align_report.txt"
                fut = ex.submit(
                    _align_one_chunk,
                    mixcr_path, preset, species, reads_layout,
                    r1, r2, out_prefix, threads_per_job, report_file
                )
                futs[fut] = str(out_prefix) + ".vdjca"

            for fut in as_completed(futs):
                try:
                    vdj = fut.result()
                    vdjca_paths.append(vdj)
                except subprocess.CalledProcessError as e:
                    errors.append(e)
                    print(e.stderr or e.output)

        if errors:
            # Surface the first meaningful error
            raise RuntimeError(f"MiXCR align failed for {len(errors)}/{n_chunks} chunks. First error: {errors[0]}")

    # --- Assemble once across ALL chunked alignments ---
    # MiXCR assemble accepts multiple .vdjca inputs; last positional is output .clns
    assemble_cmd = [mixcr_path, "assemble", "--threads", str(max(1, int(total_threads or (os.cpu_count() or 1))))] \
                   + vdjca_paths + [str(final_clns)]
    # optional report
    assemble_cmd = assemble_cmd[:3] + ["--report", str(assemble_report)] + assemble_cmd[3:]
    _run(assemble_cmd)

    # --- Export clones (single output for whole sample) ---
    export_cmd = [mixcr_path, "exportClones", str(final_clns), str(final_clones)]
    _run(export_cmd)

    print(f"[{sample}] MiXCR complete → {final_clones}")
    return {
        "clns": str(final_clns),
        "clones": str(final_clones),
        "assemble_report": str(assemble_report),
        "vdjca_list": vdjca_paths,
    }

In [22]:
# Stage data locally before starting parallel processes
patient_files_dict = {patients[0]: patient_files}
local_fastq_paths = stage_data_locally(patients, patient_files_dict, base_dir)
print(f"Locally stored paths are: {local_fastq_paths}")

Copying /content/drive/MyDrive/Bio_Informatics_RA_Assignment/Patient1/Patient1_merged_1.fastq.gz to /content/local_data/Patient1_merged_1.fastq.gz
All FASTQ files copied to local disk.
Locally stored paths are: {'Patient1': ('/content/local_data/Patient1_merged_1.fastq.gz', '/content/local_data/Patient1_merged_2.fastq.gz')}


In [23]:
r1 = local_fastq_paths[patients[0]][0]
r2 = local_fastq_paths[patients[0]][1]
print(f"r1 Path: {r1}")
print(f"r2 Path: {r2}")

r1 Path: /content/local_data/Patient1_merged_1.fastq.gz
r2 Path: /content/local_data/Patient1_merged_2.fastq.gz


In [24]:
!pip freeze > requirements.txt

In [ ]:
print("Counting reads this may take a while...")
reads = count_reads_fastq_gz(r1)
print(f"Patient1 has {reads} reads")
chunk_size = choose_chunk_size(reads)
print(f"Choosing chunk size: {chunk_size}")
# MiXCR TEST
try:
  print("Running MiXCR Test...")
  run_mixcr_pipeline(r1, r2, local_results_dir, sample="Patient1", chunk_reads=chunk_size)
except Exception as e1:
  print(f"Error During MiXCR Test for Patient1: {e1}")

Counting reads this may take a while...
Patient1 has 101745993 reads
Choosing chunk size: 5000000
Running MiXCR Test...
$ seqkit split2 -s 5000000 -O /content/local_results/chunks/Patient1_chunks_R1 /content/local_data/Patient1_merged_1.fastq.gz
$ seqkit split2 -s 5000000 -O /content/local_results/chunks/Patient1_chunks_R2 /content/local_data/Patient1_merged_2.fastq.gz
[Patient1] Chunks: 21 | parallel jobs: 1 | threads/job: 2
$ mixcr align --preset rna-seq --species hsa -OreadsLayout=Collinear --threads 2 --report /content/local_results/aligned_chunks/chunk_0001_align_report.txt /content/local_results/chunks/Patient1_chunks_R1/Patient1_merged_1.part_001.fastq.gz /content/local_results/chunks/Patient1_chunks_R2/Patient1_merged_2.part_001.fastq.gz /content/local_results/aligned_chunks/chunk_0001.vdjca
$ mixcr align --preset rna-seq --species hsa -OreadsLayout=Collinear --threads 2 --report /content/local_results/aligned_chunks/chunk_0002_align_report.txt /content/local_results/chunks/Pat